# Creating Cutouts with `astroquery.mast`'s `Cutouts` Services

## Learning Goals

By the end of this tutorial, you will:
* Learn about the 3 cutout services provided by MAST: TESSCut, HAPCut, and ZCut.
* Learn how to utilize `astroquery.mast`'s wrappers for the TESSCut and HAPCut services (`astroquery.mast.Tesscut` and `astroquery.mast.Hapcut`, respectively).
* Learn the functionalities provided by these `astroquery.mast` cutout service wrappers.
* Understand the available TESS products the `astroquery.mast.Tesscut` cutouts can be made from.

## Introduction

The Mikulski Archive for Space Telescopes (MAST) provides a suite of cutout services for several space telescope missions and use-cases:

* [<b>HAPCut Service</b>](https://mast.stsci.edu/hapcut/) : Generates cutouts out of [Hubble Advanced Products (HAPs)](https://outerspace.stsci.edu/display/HAdP/Improvements+in+HST+Astrometry) available in MAST.
* [<b>TESSCut Service</b>](https://mast.stsci.edu/tesscut/docs/index.html) : Generates cutouts out of TESS mission full frame images (namely [TESS-SPOC](https://heasarc.gsfc.nasa.gov/docs/tess/pipeline.html) and [TICA](https://archive.stsci.edu/hlsp/tica)).
* [<b>ZCut Service</b>](https://mast.stsci.edu/zcut/) : Generates cutouts out of MAST-supported deep field surveys (e.g. CANDELS, 3D-HST).

Each of these services can be accessed through a standard GET request to their respective APIs, but they can also be accessed via the `astroquery.mast` module through their respective classes: `astroquery.mast.Hapcut`, `astroquery.mast.Tesscut` and `astroquery.mast.Zcut`. In this notebook, we will walk through two examples: one where we access the TESSCut service with `astroquery.mast.Tesscut`, and one where we access the HAPCut service with `astroquery.mast.Hapcut`. A separate notebook has already been made demonstrating how to access the ZCut service with `astroquery.mast.Zcut`, and is available [here](https://github.com/spacetelescope/mast_notebooks/blob/main/notebooks/astroquery/beginner_zcut/beginner_zcut.ipynb).

The workflow for this notebook consists of:

* [Example 1: Retrieve TESS Sectors & Cutouts with `astroquery.mast.Tesscut`](#Example-1:-Retrieve-TESS-Sectors-&-Cutouts-with-astroquery.mast.Tesscut)
    * [Example 1: Get Sector Information](#Example-1:-Get-Sector-Information)
    * [Example 1: Get Cutouts](#Example-1:-Get-Cutouts)
    * [Example 1: Download Cutouts](#Example-1:-Download-Cutouts)
* [Example 2: Retrieve HAP Cutouts with `astroquery.mast.Hapcut`](#Example-2:-Retrieve-HAP-Cutouts-with-astroquery.mast.Hapcut)
    * [Example 2: Get Cutouts](#Example-2:-Get-Cutouts)
    * [Example 2: Download Cutouts](#Example-2:-Download-Cutouts)
* [ZCut for Beginners with `astroquery.mast.Zcut`](#ZCut-for-Beginners-with-astroquery.mast.Zcut)
* [Resources](#Resources)

## Imports

Below is a list of the packages used throughout this notebook, and their use-cases. Please ensure you have the recommended version of these packages installed on the environment this notebook has been launched in. See the `requirements.txt` file for recommended package versions.


* _astropy.units_ for unit conversion
* _matplotlib.pyplot_ for plotting data
* _astropy.coordinates SkyCoord_ for creating sky coordinate objects
* _astropy.io fits_ for accessing FITS files
* _astroquery.mast Hapcut_ and _Tesscut_ for retrieving and downloading the TESS FFIs


In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astroquery.mast import Hapcut, Tesscut

----

## Example 1: Retrieve TESS Sectors & Cutouts with `astroquery.mast.Tesscut`

In this example we will work with target TIC 820148 and retrieve its sector information (the TESS sectors it lands on), as well as explore the 2 ways we can retrieve cutouts of this target from the corresponding sector observations. We will also explore the two types of calibrated full frame images that we can extract cutouts from (TESS-SPOC and TICA), and how to request cutouts from one or the other.

Let's start by assigning the RA and Dec of our target to a `SkyCoord` object named `coords`.

In [ ]:
# Target TIC 820148
ra, dec = 135.068791070904, -4.93685127545661
coords = SkyCoord(ra*u.deg, dec*u.deg, frame='icrs')

### Example 1: Get Sector Information

The next thing we want to do is find the sector observations that our target lands on. To do this, we will call the `get_sectors` method. 

There are two types of products you can retrieve sector information for: the TESS-SPOC FFIs and the TICA FFIs, both available in the MAST database. These products are FFIs calibrated by two separate pipelines, and are also delivered to MAST at a slightly different cadence. The TICA products are delivered up to 3 weeks sooner than their TESS-SPOC counterparts, so users working with time-sensitive data may want to opt for setting the `product` argument to TICA to request the TICA sector information. If the `product` argument is not called, it will default to retrieve sector information for TESS-SPOC.

For this example, let's request for the sectors available for only the TICA FFIs.

In [ ]:
Tesscut.get_sectors(coordinates=coords, product='tica')

It looks like our target lands on sector 34 and 61 observations for the TICA FFIs.

**Note**: The TICA pipeline began processing sector observations from sector 27 onward, so observations for sectors 1-26 will not have any TICA cutouts available for download. Below we run the same example from above, but for TESS-SPOC products (`product` defaults to this when not explicitly called).

In [ ]:
Tesscut.get_sectors(coordinates=coords)

### Example 1: Get Cutouts

The next functionality we want to demonstrate is the `get_cutouts` functionality, which allows you to retrieve a list of unpacked FITS HDU lists for the cutouts requested of your target, for all the available sectors. This may take a while depending on how many sectors the cutouts are being retrieved from, and the size of the cutout. We recommend cutouts no larger than 30 pixels (or ~10.5 arcminutes) in either dimension. 

In the example below, we will request for `6 x 6` pixel cutouts, from all available sectors for TICA. 

In [ ]:
cutouts = Tesscut.get_cutouts(coordinates=coords, product='tica', size=6)
cutouts

Keep in mind that when the `sector` keyword argument is not explicitly called, cutout HDU lists will be generated for all the sectors available for `product`. To select for only a single sector, you can use the sector information retrieved with `get_sectors`, and declare this with the `sector` argument like so:

In [ ]:
cutouts = Tesscut.get_cutouts(coordinates=coords, product='tica', size=6, sector=34)
cutouts

And to confirm that we are retrieving cutouts from the correct sector, let's pull out the keyword value for keyword `SECTOR` in the primary header:

In [ ]:
cutouts[0][0].header['SECTOR']

Next we will plot one of the FFI cutouts for sector 34. The cutouts data is in the BinTableHDU (Extension 1), and is stored under the `FLUX` table column. Let's display the first cutout in the table:

In [ ]:
ffi_cutout = cutouts[0][1].data['FLUX'][0]

plt.imshow(ffi_cutout)

Lastly, the software used to generate this cutout is available in the primary header under the `PROCVER` keyword, as well as the product, or `FFI_TYPE`, the cutouts were made from (TICA or SPOC).

In [ ]:
print(cutouts[0][0].header.cards['PROCVER'])
print(cutouts[0][0].header.cards['FFI_TYPE'])

### Example 1: Download Cutouts

Users who want to store their requested TESS cutouts locally have the option to download them via the `download_cutouts` method. The call looks similar to the `get_cutouts` functionality, only the output product is an astropy table listing the location(s) of the downloaded cutout file(s). Users can declare the location of their download using the `path` argument; When `path` is not specified, the cutouts download in the current working directory by default. 

Let's make a `download_cutouts` request for `6 x 6` TICA cutouts of our target TIC 820148, for all sectors available for TICA:

In [ ]:
manifest = Tesscut.download_cutouts(coordinates=coords, product='tica', size=6)
manifest

Let's inspect the first cutout of the sector 34 cutout file like how we did in the `get_cutouts` example:

In [ ]:
for file in manifest['Local Path']:
    if 's0034' in file:
        ffi_cutout = fits.getdata(file)['FLUX'][0]
        
plt.imshow(ffi_cutout)

**Note**: As of June 2023, TICA is not supported by TESSCut's moving targets feature, so any call to moving targets in any of the three functions demonstrated above while `product='TICA'` will return an `InvalidQueryError`. Let's demonstrate this by requesting TICA cutouts of moving target <i>Eleonora</i>:

In [ ]:
# This cell will give an error if you uncomment the line below!

#Tesscut.download_cutouts(objectname='Eleonora', moving_target=True, product='tica', size=6)

## Example 2: Retrieve HAP Cutouts with `astroquery.mast.Hapcut`

The functionality for requesting HAP cutouts through `astroquery.mast` with HAPCut is very similar to that of TESSCut. For this example, we will request HAP cutouts for target M16 (the Eagle Nebula) observed with HST's Wide Field Camera 3 (WFC3) via program 13926.

In [ ]:
coords = SkyCoord("274.6923395574 -13.83204435743", unit='deg')

Some notable differences between `astroquery.mast` communications with the HAPCut API and the TESSCut API that you will see in the following examples are: 
* There is no `product` argument, as there is not more than one pipeline from which the HAPs are made.
* There is no `sector` argument, as sector observations are unique to the TESS mission. On that note: 
* There is no `get_sectors` functionality. Only `get_cutouts` and `download_cutouts`.

### Example 2: Get Cutouts

Let's begin with an example for `get_cutouts`. We'll request a larger cutout (`100 x 100` pixels), since the HST field of view is smaller than TESS.

In [ ]:
cutouts = Hapcut.get_cutouts(coordinates=coords, size=100)
cutouts

Let's take a look at the science array for one of the returned product cutouts. For WFC3, the science array is the `ImageHDU` in Extension 1:

In [ ]:
plt.imshow(cutouts[2][1].data)

### Example 2: Download Cutouts

Lastly, let's take a look at the `download_cutouts` functionality for the HAPCut API using the same target. This is analogous to `Tesscut.download_cutouts` and the call will look pretty much the same:

In [ ]:
manifest = Hapcut.download_cutouts(coordinates=coords, size=100)
manifest

Finally, let's plot the science results for the IR F110W observation:

In [ ]:
for file in manifest['Local Path']:
    if 'ir_f110w' in file:
        data = fits.getdata(file, 1)
        
plt.imshow(data)

## ZCut for Beginners with `astroquery.mast.Zcut`

For users who would like to make cutouts out of deep field surveys, `astroquery.mast` can make communications with the ZCut API through `astroquery.mast.Zcut`. There is an existing notebook dedicated to demonstrating this functionality, which you can find in the `mast_notebooks` GitHub repository [here](https://github.com/spacetelescope/mast_notebooks/blob/main/notebooks/astroquery/beginner_zcut/beginner_zcut.ipynb).

## Resources

The following is a list of resources that were referenced throughout the tutorial, as well as some additional references that you may find useful:

* The [astroquery.mast Documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
* The [HAPCut Service Documentation](https://mast.stsci.edu/hapcut/)
* The [TESSCut Service Documentation](https://mast.stsci.edu/tesscut/docs/index.html)
* The [TESSCut Service UI](https://mast.stsci.edu/tesscut/)
* The [ZCut Service Documentation](https://mast.stsci.edu/zcut/)
* The [ZCut For Beginners](https://github.com/spacetelescope/mast_notebooks/blob/main/notebooks/astroquery/beginner_zcut/beginner_zcut.ipynb) Notebook Tutorial
* A Description of The [Hubble Advanced Products](https://outerspace.stsci.edu/display/HAdP/Improvements+in+HST+Astrometry) (HAPs)
* The [TESS-SPOC Pipeline](https://heasarc.gsfc.nasa.gov/docs/tess/pipeline.html)
* The [TICA HLSP Pipeline](https://archive.stsci.edu/hlsp/tica)

## Citations

If you use any of astroquery's tools for published research, or any of the cutout services, please cite the authors. Follow these links for more information about citing astroquery and astrocut (the cutout services' source code):

* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)
* [Citing `astrocut`](https://ui.adsabs.harvard.edu/abs/2019ascl.soft05007B/abstract)

## About this Notebook

If you have comments or questions on this notebook, please contact us through the Archive Help Desk e-mail at archive@stsci.edu.

<b>Author(s)</b>: Jenny V. Medina <br>
<b>Keyword(s)</b>: Tutorial, TESS, TICA, SPOC, HAP, hubble, astroquery, cutouts, ffi, tesscut, hapcut, zcut <br>
<b>Last Updated</b>: Jun 2023